# Stock capital gain calculation
## First in First out law (FIFO)

### - Import libraries

In [9]:
import pandas as pd
import numpy as np

### Extract required data 

In [10]:
# Read excel file
df = pd.read_excel("tax_2023.xlsx", sheet_name="AUS_2023")

# Clean the name columns
df['Code'] = df['SYMBOL']
df['Date'] = df['DATE (US)']
df['Type'] = df['SIDE']
df['Quantity'] = df['UNIT']
df['Total Value ($)'] = df['TOTAL']

# Set a filter for neccessary data
col_list = ['Code', 'Date', 'Type', 'Quantity', 'Total Value ($)']

# Filter the dataframe to keep the data required
df = df[col_list]

# Drop NaN rows
df.dropna(inplace=True)

# Drop duplicate name rows
filter = df['Type'] != 'Type'
df = df[filter]

# Change data type from string to datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True) # Date : String to datetime format

# Sort data for Code names and event Date
df.sort_values(['Code', 'Date'], ascending=True, inplace=True) # Sorting multiple names [Code, Date]

# Reset the index numbers
df.reset_index(drop=True, inplace=True) # Reset index from 0
df['Unit Value ($)'] = df['Total Value ($)'] / df['Quantity']
df

,Code,Date,Type,Quantity,Total Value ($),Unit Value ($)
0,NDQ,2022-07-22,BUY,100,2804.00,28.040000
1,NDQ,2022-07-26,BUY,55,1505.05,27.364545
2,NDQ,2022-07-29,SELL,-100,-2845.00,28.450000
3,NDQ,2022-08-16,SELL,-55,-1655.20,30.094545
4,NDQ,2022-11-03,BUY,150,3991.50,26.610000
5,NDQ,2022-11-07,BUY,90,2343.00,26.033333
6,NDQ,2022-11-10,BUY,120,3132.60,26.105000
7,NDQ,2022-11-14,SELL,-260,-7070.10,27.192692
8,NDQ,2023-03-09,SELL,-158,-4526.70,28.650000
9,NDQ,2023-05-12,SELL,-50,-1549.00,30.980000


### Capital Gain Calculator: FIFO

In [11]:
import logging
from collections import deque
import math

class Trans:
    datetime=None
    amount=None
    price=None
 
    def __init__(self, datetime, amount, price):
        self.datetime=datetime
        self.amount=amount
        self.price=price
     
    def getInfo(self):
        return(str(self.datetime)+"; "+
                str(self.amount)+"; "+
                str(self.price))+"; "
 
def balanceFifo(all_trans):
 
    qTransactions = deque() 
    trans_result = list()
 
    for t in all_trans:
        #Add first element to the queue
        if len(qTransactions)==0:
            #logging.debug('Added the first element: %s',t.getInfo())
            qTransactions.append(t)
            continue
 
        while (t.amount!=0 and len(qTransactions)>0):
            #investigate the first element from the queue
            tq=qTransactions.popleft()
            #the same type of transaction: both sell or both buy
            if tq.amount*t.amount>0:
                #return the first element back to the same place
                qTransactions.appendleft(tq)
                #add the new element to the list
                qTransactions.append(t)
                #logging.debug('Added: %s',t.getInfo())
                break
             
            #contrary transactions: (sell and buy) or (buy and sell) 
            if tq.amount*t.amount<0:
                #logging.debug('Transaction : %s',t.getInfo())
                #logging.debug('... try to balance with: %s',tq.getInfo())
 
                #The element in the queue have more units and takes in the current transaction
                if abs(tq.amount)>abs(t.amount):
                    result = insertTransaction(tq.datetime,t.datetime,\
                            math.copysign(t.amount,tq.amount), tq.price,t.price)
                    trans_result.append(result)
                    
                    #update the amount of the element in the queue
                    tq.amount=tq.amount+t.amount
                    #return the element back to the same place
                    qTransactions.appendleft(tq)
                    #logging.debug('Removed transaction: %s',t.getInfo())
                    #the transaction has been balanced, take a new transaction
                    break
                 
                #The element from the queue and transaction have the same amount of units
                if abs(tq.amount)==abs(t.amount):
                    result = insertTransaction(tq.datetime,t.datetime,\
                                math.copysign(t.amount,tq.amount), tq.price,t.price)
                    trans_result.append(result)
                    
                    #update the amount in the transaction 
                    t.amount=0
                    #logging.debug('Balanced, removed transaction: %s',t.getInfo())
                    #logging.debug('Balanced, removed from the queue: %s',tq.getInfo())
                    #the transaction has been balanced, take a new transaction
                    continue
                    
                #The transaction has more units
                if abs(tq.amount)<abs(t.amount):
                    #update the units in transaction, (remove element from the queue)
                    t.amount=t.amount+tq.amount
                    result = insertTransaction(tq.datetime,t.datetime,tq.amount,tq.price,t.price)
                    trans_result.append(result)
                    #logging.debug('Removed from queue: %s',tq.getInfo())
                     
                    #the transaction has not been balanced, 
                    #take a new element from the queue (t.amount>0)
                    continue
                 
        #We have unbalanced transaction but the queue is empty            
        if (t.amount!=0 and len(qTransactions)==0):
            #Add unbalanced transaction to the queue
            #The queue changes polarisation
            qTransactions.append(t)
            #logging.debug('Left element: %s',t.getInfo())
     
     
    #If something remained in the queue, treat it as open or part-open transactions
    while (len(qTransactions)>0):
        tq=qTransactions.popleft()
        #logging.debug('Remained on list transaction: %s',tq.getInfo())
        
    return trans_result
 
def insertTransaction(dateStart,dateEnd,amount,priceStart,priceEnd):
    #print("Bought={}, sold={},  amount={}, buy price={}, sell_price={}, gain={}".\
    #        format(dateStart,dateEnd,amount,priceStart,priceEnd, amount*(priceEnd-priceStart)))
    result = [dateStart,dateEnd,amount,priceStart,priceEnd, amount*(priceEnd-priceStart)]
    return result

### Calculate Capital Gain for Taxation using FIFO calculator

In [12]:
# Extract code names, numbers and counts
code = df['Code'].value_counts()
print("size: ", code.size, "index: ", code.index, "value: ", code.values)

size:  2 index:  Index(['NVX', 'NDQ'], dtype='object') value:  [23 10]


In [13]:
# For loop for each Code strings 

start_date = pd.to_datetime("2022-06-30")
end_date = pd.to_datetime("2023-07-01")
sum_df = []

for j in range(0, code.size):
    # Seperate dataframe by Code name
    mask_code = df['Code'] == code.index[j]
    df_mask = df[mask_code]
    df_mask.reset_index(drop=True, inplace=True)
    print("All Transactions for", code.index[j])
    print(df_mask)
    
    # Capital gain caluculation FIFO
    trans_list=list()
    for i in range(0, df_mask.shape[0]):
        trans = Trans(df_mask['Date'][i].date(), df_mask['Quantity'][i], df_mask['Unit Value ($)'][i])
        trans_list.append(trans)
    trans_result = balanceFifo(trans_list)
    df_capital = pd.DataFrame(trans_result, columns = ['Date Purchased', 'Date Sold', 'Quantity', 'Buy Price', 'Sell Price', 'Capital Gain'])

    # Filtering sold date for 2021-2022 Capital Gain
    mask_time = (df_capital['Date Sold'] > start_date) & (df_capital['Date Sold'] < end_date)
    df_filtered = df_capital[mask_time]

    print("Financial Year Capital gain for", code.index[j])
    print(df_filtered)
    
    # Summation of capita gains (Total: any loses, discounts not included, Net: including loses or discounts)
    sum_total = np.sum(x for x in df_filtered['Capital Gain'] if x > 0)
    sum_net = np.sum(x for x in df_filtered['Capital Gain'])
    
    #For net capital gain, discounts for 'over 1 year stocks' should be applied
    diff = df_filtered['Date Sold'] - df_filtered['Date Purchased'] # How many days holding stocks
    discount = 0 # Amount of discount
    
    # If you hold stocks more than a year, you get half of discount on capital gain for taxation
    for y in range(0, diff.size):
        if (int(diff.values[y]/8.64e+13) > 365): 
            discount = discount + df_filtered['Capital Gain'][y] / 2
            sum_net = sum_net - discount
        else: 
            sum_net = sum_net
        
    print("Total capital gain for", code.index[j], ": ", round(sum_total, 2), "AUD")
    print("Net capital gain for", code.index[j], ": ", round(sum_net, 2), "AUD")
    
    # Append data into Summary dataframe
    a = [code.index[j], sum_total, sum_net]
    sum_df.append(a)

All Transactions for NVX
   Code       Date  Type  Quantity  Total Value ($)  Unit Value ($)
0   NVX 2021-12-02   BUY       100          1210.00       12.100000
1   NVX 2022-01-27   BUY       100           782.00        7.820000
2   NVX 2022-07-25  SELL      -200          -507.00        2.535000
3   NVX 2022-08-25   BUY      1000          2383.00        2.383000
4   NVX 2022-09-02   BUY      1500          3303.00        2.202000
5   NVX 2022-10-20  SELL     -2500         -6553.00        2.621200
6   NVX 2022-12-14   BUY      2000          3803.00        1.901500
7   NVX 2022-12-16   BUY      1850          3342.25        1.806622
8   NVX 2023-01-30  SELL     -1000         -1858.00        1.858000
9   NVX 2023-01-30  SELL      -500          -940.50        1.881000
10  NVX 2023-01-30  SELL      -500          -945.50        1.891000
11  NVX 2023-02-03  SELL      -850         -1630.75        1.918529
12  NVX 2023-03-06   BUY      1000          1540.00        1.540000
13  NVX 2023-03-09  SEL

C:\Users\reals\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\Users\reals\AppData\Local\Temp\ipykernel_12824\3328650654.py:31: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum_total = np.sum(x for x in df_filtered['Capital Gain'] if x > 0)
C:\Users\reals\AppData\Local\Temp\ipykernel_12824\3328650654.py:32: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum_net = np.sum(x for x in df_filtered['Capital Gain'])
C:\Us

In [14]:
# Print summary dataframe for total and net capital gain
summary = pd.DataFrame(sum_df, columns = ['Code', 'Total gain', 'Net gain'])
print(summary)
print("Total Capital Gain: ", summary['Total gain'].sum().round(), "AUD")
print("Net Capital Gain: ", summary['Net gain'].sum().round(), "AUD")

  Code  Total gain  Net gain
0  NVX  962.121622   -803.50
1  NDQ  659.150000    659.15
Total Capital Gain:  1621.0 AUD
Net Capital Gain:  -144.0 AUD


In [15]:
summary.to_excel('tax_return_2023_AUS.xlsx', sheet_name = 'AUS_CG')